In [1]:
import pandas as pd
from item_builder import build_items
from itertools import groupby
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime
import glob
from constants import currency_shards

import plotly.express as px

from fractions import Fraction
import math
from ninja import currencyreport,refine_report
from currency_conversion import conversion
from constants import hard_currency,price_in
%matplotlib inline
%load_ext autoreload
%autoreload 2


cannot import name 'ssid' from 'secrets' (/Users/swozny/.pyenv/versions/3.7.8/lib/python3.7/secrets.py)
Please set ssid in secrets.py


In [2]:
items=build_items()


https://poe.ninja/api/data/currencyoverview Fragment 200
https://poe.ninja/api/data/currencyoverview Currency 200
https://poe.ninja/api/data/itemoverview Scarab 200
https://poe.ninja/api/data/itemoverview Oil 200
https://poe.ninja/api/data/itemoverview Fossil 200
https://poe.ninja/api/data/itemoverview Resonator 200
https://poe.ninja/api/data/itemoverview Prophecy 200
https://poe.ninja/api/data/itemoverview Incubator 200
https://poe.ninja/api/data/itemoverview UniqueMap 200
https://poe.ninja/api/data/itemoverview UniqueJewel 200
https://poe.ninja/api/data/itemoverview UniqueFlask 200
https://poe.ninja/api/data/itemoverview UniqueArmour 200
https://poe.ninja/api/data/itemoverview UniqueWeapon 200
https://poe.ninja/api/data/itemoverview UniqueAccessory 200
https://poe.ninja/api/data/itemoverview Essence 200
https://poe.ninja/api/data/itemoverview DeliriumOrb 200
https://poe.ninja/api/data/itemoverview DivinationCard 200
https://poe.ninja/api/data/itemoverview Map 200
https://poe.ninja/ap

Could not find an item type for Chromium Valdo's Rest Watchstone of Influence
Could not find an item type for Titanium Lira Arthain Watchstone
Could not find an item type for Titanium Glennach Cairns Watchstone
Could not find an item type for Chromium Valdo's Rest Watchstone
Could not find an item type for Chromium Glennach Cairns Watchstone
Could not find an item type for Chromium Valdo's Rest Watchstone
Could not find an item type for Chromium Haewark Hamlet Watchstone
Could not find an item type for Hardening Chromium Glennach Cairns Watchstone of Wealth
Could not find an item type for Delver's Titanium Glennach Cairns Watchstone
Could not find an item type for Titanium Lira Arthain Watchstone
Could not find an item type for Chromium Glennach Cairns Watchstone
Could not find an item type for Chromium Haewark Hamlet Watchstone
Could not find an item type for Titanium Glennach Cairns Watchstone
Could not find an item type for Chromium Glennach Cairns Watchstone
Could not find an item 

Couldn't find match for Earthquake 1 23 Corrupted Using Earthquake, 2 0 instead
Couldn't find match for Chance to Bleed Support 1 23 Corrupted Using Chance to Bleed Support, 3 0 instead
Couldn't find match for Discipline 18 0  Using Discipline, 1 0 instead
Couldn't find match for Onslaught Support 12 0  Using Onslaught Support, 3 0 instead
Couldn't find match for Ancestral Protector 16 0  Using Ancestral Protector, 7 0 instead
Couldn't find match for Faster Casting Support 16 0  Using Faster Casting Support, 3 0 instead
Couldn't find match for Assassin's Mark 18 0  Using Assassin's Mark, 1 0 instead
Couldn't find match for Vaal Flameblast 1 0 Corrupted Using Vaal Flameblast, 20 0 instead
Couldn't find match for Conductivity 16 0  Using Conductivity, 6 0 instead
Couldn't find match for Flammability 19 0  Using Flammability, 7 0 instead
Couldn't find match for Assassin's Mark 19 0  Using Assassin's Mark, 1 0 instead
Couldn't find match for Vaal Summon Skeletons 1 0 Corrupted Using Vaal S

In [3]:

def save_currency_report(cr):
    now= datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%s')
    cr.to_csv(f'currency_reports/{now}.csv')
def sunburst(inventory):
    new=inventory.groupby(['type','type_line']).sum().reset_index()
    fig =px.sunburst(
        new,
        path=['type', 'type_line'], values='chaos_value',
        template='plotly_dark'
    )
    fig.show()
cr = currencyreport()
save_currency_report(cr)
cr=refine_report(cr)
cr=cr.set_index('get_currency')
cr=cr[cr['receive']==False]


https://poe.ninja/api/data/currencyoverview Fragment 200
https://poe.ninja/api/data/currencyoverview Currency 200


In [4]:
[item for item in items if item.name=='The Warden' ]

[]

In [5]:
df=pd.DataFrame([dict(chaos_value=item.value,type=item.type,type_line=item.type_line,price=item.price,shoplink_template=item.shoplink_template,stack_size=item.stack_size) for item in items])
df.loc[df.type_line.isin(hard_currency),'type']='Hard Currency'
df.loc[df.type_line=='Chaos Orb','chaos_value']=df.loc[df.type_line=='Chaos Orb','stack_size']
inventory=df[~df['type'].isin([None])&(df.chaos_value>0)]
worthless=df[(df.chaos_value==0)&df.type.str.contains('Uniq')]


In [ ]:
sunburst(inventory)

In [ ]:
notcurrency=inventory.groupby('type').sum().sort_values(ascending=False,by='chaos_value')
currency=inventory[inventory.type.isin(['Currency','Hard Currency'])].groupby('type_line').sum()['chaos_value'].reset_index()
currency.loc[currency.chaos_value<5,'type_line']='other'
currency=currency.groupby('type_line').sum().sort_values(ascending=False,by='chaos_value')


In [ ]:
total_on_offer=inventory#[~inventory.type.isin(['Currency','Hard Currency'])]

In [ ]:
total_on_offer.groupby('type').sum().sort_values(ascending=False,by='chaos_value')


In [ ]:
print(f"Liquidating a total of {total_on_offer.sum()['chaos_value']}")
print(f"Currency total {currency.sum()['chaos_value']}")

In [ ]:
fig = px.bar(notcurrency['chaos_value'],template='plotly_dark',title='Total Breakdown')
fig.show()

In [ ]:
cr['value']['Chaos Orb']=1
exchange=cr['value']

In [ ]:
register = inventory.apply(lambda x:pd.concat([x, x.chaos_value/exchange],axis=0),axis=1).set_index(['chaos_value','type','type_line','price','shoplink_template','stack_size'])
register=register.stack().reset_index()
register.columns = ['chaos_value','type','type_line','chaos_price','shoplink_template','stack_size','fx','value_fx']
register=register[(register.chaos_value/register.stack_size)>0.75]

In [ ]:
register=register[register.fx=='Chaos Orb']

In [ ]:
register

In [ ]:
def keywords(words,mult):
    return lambda item: mult if any(word in item.type_line for word in words) else 1
onefive=["Fragment of","Breachstone",'inged']
def up_price(item):
    base =1.3
    if item.type_line=='Offering to the Goddess':
        base=base
    if item.type in ('SkillGem','Base','Prophecy'):
        base=1
    base*=keywords(onefive,1.35)(item)
    if item.type=='Map':
        base*= 1.2
    if item.type=='Incubator':
        base*= 1.5
    if item.type =="Catalyst" and item.stack_size <10:
        base*=5
    else:
        base*=1
    if item.stack_size>50:
        base*=1.1

    return math.ceil(base*item.value_fx)

In [ ]:
weak_currency = ['Orb of Alteration','Orb of Alchemy']
register['up_priced_value_fx']=register.apply(up_price,axis=1)
register['final_price']=register.apply(lambda self: Fraction(self.up_priced_value_fx/self.stack_size).limit_denominator(self.stack_size),axis=1)
register['offered_value_chaos']=register.apply(lambda self: float(self.final_price)*exchange[self.fx]*self.stack_size,axis=1)
register['fx_id']=register.fx.map(conversion)
register['difference_fx']=register.offered_value_chaos- register.chaos_value

In [ ]:
# register=register[(register.difference_fx/register.chaos_value)<0.1]
register=register[register['type']!='Hard Currency']
register = register[register.chaos_value>1]


In [ ]:
columns_to_drop_for_visibility=['shoplink_template','up_priced_value_fx','value_fx']
preview = register.drop(columns_to_drop_for_visibility,axis=1)
preview.sort_values('difference_fx').tail(10)


In [ ]:
preview[preview.type_line=="Polished Breach Scarab"]

In [ ]:
_register = register[register['type']!='Hard Currency']
_register=_register[_register.final_price>1]
_register=_register[~_register.type_line.isin(currency_shards)]
content= list(_register.apply(lambda row: row.shoplink_template.format(**row),axis=1))


In [ ]:
assert len(str(content))< 150_000
len(str(content))

In [ ]:
print("Liquidizing")
df['datetime']=datetime.datetime.now()
df.to_csv(f'reports/{datetime.datetime.now()}.csv')

In [ ]:

dfs=pd.concat([pd.read_csv(file_type_line) for file_type_line in glob.glob('reports/*.csv')])
#dfs[
hc=dfs.type_line.isin(['Orb of Alteration','Orb of Alchemy','Chaos Orb','Vaal Orb','Divine Orb','Exalted Orb','Mirror Shard','Stacked Deck','Mirror of Kalandra'])
dfs.loc[hc,'type']='Hard Currency'
dfs[dfs.type=='Hard Currency'].sort_values('datetime').tail(10)


In [ ]:
import datetime
_dfs=dfs.groupby(['type','datetime']).sum().reset_index().sort_values('type')
fig = px.area(_dfs.reset_index(), x="datetime", y="chaos_value",template='plotly_dark', color="type",category_orders={'Hard Currency':1,'Currency':0})
fig.show()

In [ ]:
_dfs=dfs.groupby(['type','datetime']).sum().reset_index().sort_values('type')
_dfs=_dfs[pd.to_datetime(_dfs['datetime'])>datetime.datetime.now()-datetime.timedelta(hours=24)]
fig = px.area(_dfs.reset_index(), x="datetime", y="chaos_value",template='plotly_dark', color="type",category_orders={'Hard Currency':1,'Currency':0})
fig.show()

In [ ]:

tmp = dfs.groupby(['datetime']).sum().reset_index()
tmp['datetime']=pd.to_datetime(tmp['datetime'])
tmp=tmp.set_index('datetime')
slope = pd.Series(np.gradient(tmp['chaos_value'].values), tmp.index, name='slope')

slope.plot()

In [ ]:
dfs=pd.concat([pd.read_csv(file_type_line) for file_type_line in glob.glob('temp/*.csv')])
dfs[dfs.type=='Hard Currency'].sort_values(['type_line'])


In [ ]:
from forum_updater import update_forum
import time
print(len(str(content)))

assert len(str(content))<49_999

In [ ]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

In [ ]:
_content=[content]#,content[int(l/3):int(2*l/3)],content[int(2*l/3):]]

update_forum("\n".join(_content[0])[:49999],'3194663')
# update_forum('asd','3096135')https://www.pathofexile.com/forum/view-thread/3194663